In [268]:
#import environments
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [269]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [270]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [271]:
url = 'https://www.indeed.com/salaries'
browser.visit(url)
time.sleep(1)

In [272]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

In [273]:
job_titles = ['junior data analyst', 'data analyst', 'senior data analyst', 
              'data engineer', 'data scientist', 
              'senior data scientist', 'Business intelligence analyst',
              "Chief information officer", "Business Systems Analyst",
             "Data Warehouse Architect", "Software Architect",
             "Database manager","Senior Data modeler", "Database developer", 
              "Database administrator", "Machine learning engineer"]

# Test Case - try specific title
#job_titles = ["Data modeler"]

salaries_by_job_title = []
salary_type_by_job_title = []
salary_description = []
source_urls = []


In [274]:
for job in job_titles:
    try:
        print(job)
        try:
            browser.find_by_id('cmp-salary-search-input').first.fill(job)
        except:
            try:
                browser.find_by_id('cmp-salary-search-input-title').first.fill(job)
            except:
                pass
        time.sleep(1)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
#         print(soup.prettify())
        search_results = soup.find('div', class_='cmp-salary-search-results')
#         print(search_results)
#         print('------------')
        keywords = search_results.find_all('a', class_='cmp-salary-search-result-title')
#         print(keywords)
#         print('------------')
        for keyword in keywords:
            print(keyword.text)
        print('------------')
        #option = browser.find_by_partial_text('data analyst').mouse_over()
        option = browser.links.find_by_partial_text(job)
        option.mouse_over()
        print(option.text)
        browser.click_link_by_id('cmp-salary-search-submit')
        time.sleep(1)
        print(browser.url)
        source_urls.append(browser.url)
        browser.visit(browser.url)
        time.sleep(1)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        #print(soup.prettify())
        try:
            print('-----Trying Results Page type 1-------')
            avg_salary = soup.find('div', class_='sal-agg-nonbase__average-salary-value').text
            avg_salary_type = soup.find('div', class_='sal-agg-nonbase__average-salary-type').text   
            avg_salary_summary = soup.find('h3', class_='sal-agg-nonbase__salary-explain-summary').text
            salaries_by_job_title.append(avg_salary)
            salary_type_by_job_title.append(avg_salary_type)
            salary_description.append(avg_salary_summary)
            print('-----Completed Results Page type 1-------')
        except:
            print('----- Trying Results Page type 2-------')
            avg_salary = soup.find('strong', class_='cmp-salary-amount').text           
            avg_salary_type = soup.find('div', class_='cmp-sal-salary').text 
            avg_salary_type_array = avg_salary_type.split('per') 
            avg_salary_type = f'per{avg_salary_type_array[1]}'
            avg_salary_summary = soup.find('div', class_='cmp-salary-blurb-text').text  
            salaries_by_job_title.append(avg_salary)
            salary_type_by_job_title.append(avg_salary_type)
            salary_description.append(avg_salary_summary)
            print('----- Completed Results Page type 2-------')
        
        print(avg_salary)
        print(avg_salary_type)
        print(avg_salary_summary)
        browser.back()
        time.sleep(1)
    except AttributeError as e:
        print(f'ERROR: {e}')
        pass
        

junior data analyst
Junior Data Analyst
------------
Popular salaries for junior data analyst
https://www.indeed.com/career/junior-data-analyst/salaries
-----Trying Results Page type 1-------
-----Completed Results Page type 1-------
$57,236
per year
The average salary for a junior data analyst  is $57,236 per year in the United States. 
data analyst
Data Entry Clerk
Data Analyst
Senior Data Analyst
Data Analyst Manager
Junior Data Analyst
Entry Level Data Analyst
------------
Popular salaries for data analyst
https://www.indeed.com/career/data-analyst/salaries
-----Trying Results Page type 1-------
-----Completed Results Page type 1-------
$75,121
per year
The average salary for a data analyst  is $75,121 per year in the United States and  $2,500 cash bonus per year.
senior data analyst
Senior Data Analyst
------------
Popular salaries for senior data analyst
https://www.indeed.com/career/senior-data-analyst/salaries
-----Trying Results Page type 1-------
-----Completed Results Page t

In [275]:
browser.quit()

In [276]:
salary_table = pd.DataFrame({"Job Title":job_titles,
                             "Avg Salary":salaries_by_job_title,
                             "Salary Type": avg_salary_type,
                             "Salary Description": salary_description
                             })
salary_table

,Job Title,Avg Salary,Salary Type,Salary Description
0,junior data analyst,"$57,236",per year,The average salary for a junior data analyst ...
1,data analyst,"$75,121",per year,"The average salary for a data analyst is $75,..."
2,senior data analyst,"$95,984",per year,The average salary for a senior data analyst ...
3,data engineer,"$129,421",per year,The average salary for a data engineer is $12...
4,data scientist,"$121,691",per year,The average salary for a data scientist is $1...
5,senior data scientist,"$150,981",per year,The average salary for a senior data scientist...
6,Business intelligence analyst,"$95,766",per year,The average salary for a business intelligence...
7,Chief information officer,"$129,724",per year,The average salary for a chief information off...
8,Business Systems Analyst,"$90,587",per year,The average salary for a business systems anal...
9,Data Warehouse Architect,"$134,214",per year,The average salary for a Data Warehouse Archi...


In [277]:
salary_table.to_csv(r'avg_salary_by_title.csv', index = False)